In [1]:
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


df = pd.read_excel('smFISH_screen_annotations.xlsx', header=[2])
df = df.iloc[1:201, 1:11]

## Importing the scoring

In [4]:
df_VNC_1 = pd.read_csv('scoringResults/VNC/rater1.csv')
df_VNC_2 = pd.read_csv('scoringResults/VNC/rater2.csv')
df_VNC_3 = pd.read_csv('scoringResults/VNC/rater3.csv')

df_NMJ_1 = pd.read_csv('scoringResults/NMJ/rater1.csv')
df_NMJ_2 = pd.read_csv('scoringResults/NMJ/rater2.csv')
df_NMJ_3 = pd.read_csv('scoringResults/NMJ/rater3.csv')

df_CB_1 = pd.read_csv('scoringResults/CB/rater1.csv')
df_CB_2 = pd.read_csv('scoringResults/CB/rater2.csv')
df_CB_3 = pd.read_csv('scoringResults/CB/rater3.csv')

df_MB_1 = pd.read_csv('scoringResults/MB/rater1.csv')
df_MB_2 = pd.read_csv('scoringResults/MB/rater2.csv')
df_MB_3 = pd.read_csv('scoringResults/MB/rater3.csv')

df_OL_1 = pd.read_csv('scoringResults/OL/rater1.csv')
df_OL_2 = pd.read_csv('scoringResults/OL/rater2.csv')
df_OL_3 = pd.read_csv('scoringResults/OL/rater3.csv')

df_nerve_1 = pd.read_csv('scoringResults/nerve/rater1.csv')
df_nerve_2 = pd.read_csv('scoringResults/nerve/rater2.csv')
df_nerve_3 = pd.read_csv('scoringResults/nerve/rater3.csv')



## Making dataframes comparable

1. function threeway_comparer 
    * Make sure the dataframes are equal, e.g. keeping common columns and common figures
    * Removing "pattern" questions as they take multiple answers

In [5]:
def threeway_comparer(df1, df2, df3, column_figures, columns_string, columns_drop=None):
    if columns_drop is None:
        columns_drop = []
    pass
    
    df_common = df1.merge(df2, on=column_figures, how='inner')
    common_index = list(df_common.merge(df3, on=column_figures, how='inner')[column_figures])
    common_columns = list((df2.columns.intersection(df1.columns)).intersection(df3.columns))
    
    df_1 = df1[common_columns].set_index(column_figures)
    df_1 = df_1[df_1.index.isin(common_index)].sort_index()
    df_1.fillna('Non applicable / Missing Image', inplace=True)

    df_2 = df2[common_columns].set_index(column_figures)
    df_2 = df_2[df_2.index.isin(common_index)].sort_index()
    df_2.fillna('Non applicable / Missing Image', inplace=True)

    df_3 = df3[common_columns].set_index(column_figures)
    df_3 = df_3[df_3.index.isin(common_index)].sort_index()
    df_3.fillna('Non applicable / Missing Image', inplace=True)

    common_columns.remove(column_figures)
    columns_numeric = list(set(common_columns)^set(columns_string)^set(columns_drop))

    return df_1[columns_numeric], df_2[columns_numeric], df_3[columns_numeric]

## Defining functions for Inter Observer Agreement

#### Simple agreement (per question)

In [6]:
def simple_agreement(df1, df2):
    """Computes simple agreement for pair-wise annotators.
    :param ann1: annotations dataframe provided by first annotator
    :type ann1: df
    :param ann2: annotations dataframe provided by second annotator
    :type ann2: df
    :rtype: float
    :return: percentage of simple agreement
    """

    simple_agreement_question = []
    
    for i in range(0, df1.shape[1]):
        ann1 = df1.T.to_numpy()[i].tolist()
        ann2 = df2.T.to_numpy()[i].tolist()

        count = 0
        for an1, an2 in zip(ann1, ann2):
            if an1 == an2:
                count += 1
        A = count / len(ann1)  # observed agreement A (Po)

        simple_agreement_question.append(A)
        
    df_simple_agreement = pd.DataFrame(simple_agreement_question, index= df1.columns, columns= ['Simple Agreement'])#.T
    
    return df_simple_agreement

#### Cohen Kappa (per dataframe)

In [7]:
def cohen_kappa(df1, df2):
    """Computes Cohen kappa for pair-wise annotators.
    :param ann1: annotations dataframe provided by first annotator
    :type ann1: df
    :param ann2: annotations dataframe provided by second annotator
    :type ann2: df
    :rtype: float
    :return: Cohen kappa statistic
    
    cite: https://towardsdatascience.com/inter-annotator-agreement-2f46c6d37bf3
    """
    ann1 = df1.to_numpy().flatten().tolist()
    ann2 = df2.to_numpy().flatten().tolist()

    count = 0
    for an1, an2 in zip(ann1, ann2):
        if an1 == an2:
            count += 1
    A = count / len(ann1)  # observed agreement A (Po)

    uniq = set(ann1 + ann2)
    E = 0  # expected agreement E (Pe)
    for item in uniq:
        cnt1 = ann1.count(item)
        cnt2 = ann2.count(item)
        count = ((cnt1 / len(ann1)) * (cnt2 / len(ann2)))
        E += count

    return round((A - E) / (1 - E), 4)

#### Cohen Kappa (per question)

In [8]:
def cohen_kappa_que(df1, df2):
    """Computes Cohen kappa for pair-wise annotators for each question.
    :param ann1: annotations dataframe provided by first annotator
    :type ann1: df
    :param ann2: annotations dataframe provided by second annotator
    :type ann2: df
    :type: float
    :return: Cohen kappa statistic
    
    cite: https://towardsdatascience.com/inter-annotator-agreement-2f46c6d37bf3
    """
    cohen_kappa_question = []
    
    for i in range(0, df1.shape[1]):
        ann1 = df1.T.to_numpy()[i].tolist()
        ann2 = df2.T.to_numpy()[i].tolist()

        count = 0
        for an1, an2 in zip(ann1, ann2):
            if an1 == an2:
                count += 1
        A = count / len(ann1)  # observed agreement A (Po)

        uniq = set(ann1 + ann2)
        E = 0  # expected agreement E (Pe)
        for item in uniq:
            cnt1 = ann1.count(item)
            cnt2 = ann2.count(item)
            count = ((cnt1 / len(ann1)) * (cnt2 / len(ann2)))
            E += count

        cohen_kappa_question.append(round((A - E) / (1 - E), 4))
        
    df_cohen_k = pd.DataFrame(cohen_kappa_question, index= df1.columns, columns= ['Cohen Kappa'])#.T
    
    return df_cohen_k #cohen_kappa_question

# Mushroom Body

In [9]:
columns_drop_MB = ['General comments', 
                   'Flag this gene as a potential one to use for a figure in the paper?',
                   'Is figure quality too poor?', 
                   'Nascent transcript level', 
                   'Is there obvious RNA/protein discordance (whole image)?', 
                   'Cytosolic RNA level', 
                   'What is the level of protein expression?', 
                   'What is the level of RNA expression?']


df_1MB, df_2MB, df_3MB = threeway_comparer(df_MB_1, df_MB_2,  df_MB_3, 'figure_id', 
                         ['If there is RNA expression, what patterns does the RNA make?',
                          'If there is protein expression, what patterns does the protein make?'], columns_drop_MB)

dfaMB = (cohen_kappa_que(df_2MB, df_3MB))
dfbMB = (cohen_kappa_que(df_2MB, df_1MB))
dfcMB = (cohen_kappa_que(df_3MB, df_1MB))

dfMB_mean = (dfaMB + dfbMB + dfcMB)/3


df_simple_aMB = simple_agreement(df_3MB, df_2MB)
df_simple_bMB = simple_agreement(df_1MB, df_2MB)
df_simple_cMB = simple_agreement(df_1MB, df_3MB)

dfMB_simple_mean = (df_simple_aMB + df_simple_bMB + df_simple_cMB)/3

In [10]:
df_MB_agreement = df_1MB.copy()

for column in df_MB_agreement.columns:

    df_MB_agreement[column] = np.where(((df_1MB[column] != df_2MB[column]) & (df_1MB[column] == df_3MB[column])), 2, df_MB_agreement[column])
    df_MB_agreement[column] = np.where(((df_1MB[column] != df_2MB[column]) & (df_2MB[column] == df_3MB[column])), 2, df_MB_agreement[column])

    df_MB_agreement[column] = np.where(((df_2MB[column] != df_3MB[column]) & (df_2MB[column] == df_1MB[column])), 2, df_MB_agreement[column])
    df_MB_agreement[column] = np.where(((df_2MB[column] != df_3MB[column]) & (df_3MB[column] == df_1MB[column])), 2, df_MB_agreement[column])

    df_MB_agreement[column] = np.where(((df_1MB[column] != df_3MB[column]) & (df_1MB[column] == df_2MB[column])), 2, df_MB_agreement[column])
    df_MB_agreement[column] = np.where(((df_1MB[column] != df_3MB[column]) & (df_3MB[column] == df_2MB[column])), 2, df_MB_agreement[column])


    df_MB_agreement[column] = np.where(((df_1MB[column] == df_2MB[column]) & (df_3MB[column] == df_2MB[column]) & (df_3MB[column] == df_1MB[column])), 3, df_MB_agreement[column])
    df_MB_agreement[column] = np.where(((df_1MB[column] != df_2MB[column]) & (df_3MB[column] != df_2MB[column]) & (df_3MB[column] != df_1MB[column])), 0, df_MB_agreement[column])

#df_MB_agreement

# Central Brain
## Agreement values

In [11]:
columns_drop_CB = ['General comments', 
                   'Flag this gene as a potential one to use for a figure in the paper?',
                   'Is figure quality too poor?', 
                   'Nascent transcript level', 
                   'Is there obvious RNA/protein discordance (whole image)?', 
                   'Cytosolic RNA level', 
                   'What is the level of protein expression?', 
                   'What is the level of RNA expression?']


df_1CB, df_2CB, df_3CB = threeway_comparer(df_CB_1, df_CB_2,  df_CB_3, 'figure_id', 
                         ['If there is RNA expression, what patterns does the RNA make?',
                          'If there is protein expression, what patterns does the protein make?'], columns_drop_CB)

dfaCB = (cohen_kappa_que(df_2CB, df_3CB))
dfbCB = (cohen_kappa_que(df_2CB, df_1CB))
dfcCB = (cohen_kappa_que(df_3CB, df_1CB))

dfCB_mean = (dfaCB + dfbCB + dfcCB)/3


df_simple_aCB = simple_agreement(df_3CB, df_2CB)
df_simple_bCB = simple_agreement(df_1CB, df_2CB)
df_simple_cCB = simple_agreement(df_1CB, df_3CB)

dfCB_simple_mean = (df_simple_aCB + df_simple_bCB + df_simple_cCB)/3

## Agreement matrix

In [13]:
df_CB_agreement = df_1CB.copy()

for column in df_CB_agreement.columns:

    df_CB_agreement[column] = np.where(((df_1CB[column] != df_2CB[column]) & (df_1CB[column] == df_3CB[column])), 2, df_CB_agreement[column])
    df_CB_agreement[column] = np.where(((df_1CB[column] != df_2CB[column]) & (df_2CB[column] == df_3CB[column])), 2, df_CB_agreement[column])

    df_CB_agreement[column] = np.where(((df_2CB[column] != df_3CB[column]) & (df_2CB[column] == df_1CB[column])), 2, df_CB_agreement[column])
    df_CB_agreement[column] = np.where(((df_2CB[column] != df_3CB[column]) & (df_3CB[column] == df_1CB[column])), 2, df_CB_agreement[column])

    df_CB_agreement[column] = np.where(((df_1CB[column] != df_3CB[column]) & (df_1CB[column] == df_2CB[column])), 2, df_CB_agreement[column])
    df_CB_agreement[column] = np.where(((df_1CB[column] != df_3CB[column]) & (df_3CB[column] == df_2CB[column])), 2, df_CB_agreement[column])

    df_CB_agreement[column] = np.where(((df_1CB[column] == df_2CB[column]) & (df_3CB[column] == df_2CB[column]) & (df_3CB[column] == df_1CB[column])), 3, df_CB_agreement[column])
    df_CB_agreement[column] = np.where(((df_1CB[column] != df_2CB[column]) & (df_3CB[column] != df_2CB[column]) & (df_3CB[column] != df_1CB[column])), 0, df_CB_agreement[column])

#df_CB_agreement

# Ventral Nerve Cord

In [14]:
columns_drop_VNC = ['General comments', 
                   'Flag this gene as a potential one to use for a figure in the paper?',
                   'Is figure quality too poor?', 
                   'Nascent transcript level', 
                   'Is there obvious RNA/protein discordance (whole image)?', 
                   'Cytosolic RNA level', 
                   'What is the level of protein expression?', 
                   'What is the level of RNA expression?']


df_1VNC, df_2VNC, df_3VNC = threeway_comparer(df_VNC_1, df_VNC_2,  df_VNC_3, 'figure_id', 
                         ['If there is RNA expression, what patterns does the RNA make?',
                          'If there is protein expression, what patterns does the protein make?'], columns_drop_VNC)

dfaVNC = (cohen_kappa_que(df_2VNC, df_3VNC))
dfbVNC = (cohen_kappa_que(df_2VNC, df_1VNC))
dfcVNC = (cohen_kappa_que(df_3VNC, df_1VNC))

dfVNC_mean = (dfaVNC + dfbVNC + dfcVNC)/3


df_simple_aVNC = simple_agreement(df_3VNC, df_2VNC)
df_simple_bVNC = simple_agreement(df_1VNC, df_2VNC)
df_simple_cVNC = simple_agreement(df_1VNC, df_3VNC)

dfVNC_simple_mean = (df_simple_aVNC + df_simple_bVNC + df_simple_cVNC)/3

In [15]:
df_VNC_agreement = df_1VNC.copy()

for column in df_VNC_agreement.columns:

    df_VNC_agreement[column] = np.where(((df_1VNC[column] != df_2VNC[column]) & (df_1VNC[column] == df_3VNC[column])), 2, df_VNC_agreement[column])
    df_VNC_agreement[column] = np.where(((df_1VNC[column] != df_2VNC[column]) & (df_2VNC[column] == df_3VNC[column])), 2, df_VNC_agreement[column])

    df_VNC_agreement[column] = np.where(((df_2VNC[column] != df_3VNC[column]) & (df_2VNC[column] == df_1VNC[column])), 2, df_VNC_agreement[column])
    df_VNC_agreement[column] = np.where(((df_2VNC[column] != df_3VNC[column]) & (df_3VNC[column] == df_1VNC[column])), 2, df_VNC_agreement[column])

    df_VNC_agreement[column] = np.where(((df_1VNC[column] != df_3VNC[column]) & (df_1VNC[column] == df_2VNC[column])), 2, df_VNC_agreement[column])
    df_VNC_agreement[column] = np.where(((df_1VNC[column] != df_3VNC[column]) & (df_3VNC[column] == df_2VNC[column])), 2, df_VNC_agreement[column])

    df_VNC_agreement[column] = np.where(((df_1VNC[column] == df_2VNC[column]) & (df_3VNC[column] == df_2VNC[column]) & (df_3VNC[column] == df_1VNC[column])), 3, df_VNC_agreement[column])
    df_VNC_agreement[column] = np.where(((df_1VNC[column] != df_2VNC[column]) & (df_3VNC[column] != df_2VNC[column]) & (df_3VNC[column] != df_1VNC[column])), 0, df_VNC_agreement[column])

#df_VNC_agreement

# Neuromuscular Junction

In [16]:
columns_drop_NMJ = ['General comments', 
                   'Flag this gene as a potential one to use for a figure in the paper?',
                   'Is figure quality too poor?', 
                   'Nascent transcript level', 
                   'Is there obvious RNA/protein discordance (whole image)?', 
                   'Cytosolic RNA level', 
                   'What is the level of protein expression?', 
                   'What is the level of RNA expression?', 
                    'Is protein distribution cell-type specific?', 
                    'Is RNA distribution cell-type specific?']


df_1NMJ, df_2NMJ, df_3NMJ = threeway_comparer(df_NMJ_1, df_NMJ_2,  df_NMJ_3, 'figure_id', 
                         ['What patterns does the RNA make?',
                          'What patterns does the protein make?'], columns_drop_NMJ)

dfaNMJ = (cohen_kappa_que(df_2NMJ, df_3NMJ))
dfbNMJ = (cohen_kappa_que(df_2NMJ, df_1NMJ))
dfcNMJ = (cohen_kappa_que(df_3NMJ, df_1NMJ))

dfNMJ_mean = (dfaNMJ + dfbNMJ + dfcNMJ)/3


df_simple_aNMJ = simple_agreement(df_3NMJ, df_2NMJ)
df_simple_bNMJ = simple_agreement(df_1NMJ, df_2NMJ)
df_simple_cNMJ = simple_agreement(df_1NMJ, df_3NMJ)

dfNMJ_simple_mean = (df_simple_aNMJ + df_simple_bNMJ + df_simple_cNMJ)/3

In [17]:
df_NMJ_agreement = df_1NMJ.copy()

for column in df_NMJ_agreement.columns:

    df_NMJ_agreement[column] = np.where(((df_1NMJ[column] != df_2NMJ[column]) & (df_1NMJ[column] == df_3NMJ[column])), 2, df_NMJ_agreement[column])
    df_NMJ_agreement[column] = np.where(((df_1NMJ[column] != df_2NMJ[column]) & (df_2NMJ[column] == df_3NMJ[column])), 2, df_NMJ_agreement[column])

    df_NMJ_agreement[column] = np.where(((df_2NMJ[column] != df_3NMJ[column]) & (df_2NMJ[column] == df_1NMJ[column])), 2, df_NMJ_agreement[column])
    df_NMJ_agreement[column] = np.where(((df_2NMJ[column] != df_3NMJ[column]) & (df_3NMJ[column] == df_1NMJ[column])), 2, df_NMJ_agreement[column])

    df_NMJ_agreement[column] = np.where(((df_1NMJ[column] != df_3NMJ[column]) & (df_1NMJ[column] == df_2NMJ[column])), 2, df_NMJ_agreement[column])
    df_NMJ_agreement[column] = np.where(((df_1NMJ[column] != df_3NMJ[column]) & (df_3NMJ[column] == df_2NMJ[column])), 2, df_NMJ_agreement[column])

    df_NMJ_agreement[column] = np.where(((df_1NMJ[column] == df_2NMJ[column]) & (df_3NMJ[column] == df_2NMJ[column]) & (df_3NMJ[column] == df_1NMJ[column])), 3, df_NMJ_agreement[column])
    df_NMJ_agreement[column] = np.where(((df_1NMJ[column] != df_2NMJ[column]) & (df_3NMJ[column] != df_2NMJ[column]) & (df_3NMJ[column] != df_1NMJ[column])), 0, df_NMJ_agreement[column])

#df_NMJ_agreement

# Optic Lobe

In [18]:
columns_drop_OL = ['General comments', 
                   'Flag this gene as a potential one to use for a figure in the paper?',
                   'Nascent transcript level', 
                   'Is there obvious RNA/protein discordance (whole image)?', 
                   'Cytosolic RNA level', 
                   'What is the level of protein expression?', 
                   'What is the level of RNA expression?']


df_1OL, df_2OL, df_3OL = threeway_comparer(df_OL_1, df_OL_2,  df_OL_3, 'figure_id', 
                         ['If there is RNA expression, what patterns does the RNA make?',
                          'If there is protein expression, what patterns does the protein make?'], columns_drop_OL)

dfaOL = (cohen_kappa_que(df_2OL, df_3OL))
dfbOL = (cohen_kappa_que(df_2OL, df_1OL))
dfcOL = (cohen_kappa_que(df_3OL, df_1OL))

dfOL_mean = (dfaOL + dfbOL + dfcOL)/3


df_simple_aOL = simple_agreement(df_3OL, df_2OL)
df_simple_bOL = simple_agreement(df_1OL, df_2OL)
df_simple_cOL = simple_agreement(df_1OL, df_3OL)

dfOL_simple_mean = (df_simple_aOL + df_simple_bOL + df_simple_cOL)/3

In [19]:
df_OL_agreement = df_1OL.copy()

for column in df_OL_agreement.columns:

    df_OL_agreement[column] = np.where(((df_1OL[column] != df_2OL[column]) & (df_1OL[column] == df_3OL[column])), 2, df_OL_agreement[column])
    df_OL_agreement[column] = np.where(((df_1OL[column] != df_2OL[column]) & (df_2OL[column] == df_3OL[column])), 2, df_OL_agreement[column])

    df_OL_agreement[column] = np.where(((df_2OL[column] != df_3OL[column]) & (df_2OL[column] == df_1OL[column])), 2, df_OL_agreement[column])
    df_OL_agreement[column] = np.where(((df_2OL[column] != df_3OL[column]) & (df_3OL[column] == df_1OL[column])), 2, df_OL_agreement[column])

    df_OL_agreement[column] = np.where(((df_1OL[column] != df_3OL[column]) & (df_1OL[column] == df_2OL[column])), 2, df_OL_agreement[column])
    df_OL_agreement[column] = np.where(((df_1OL[column] != df_3OL[column]) & (df_3OL[column] == df_2OL[column])), 2, df_OL_agreement[column])

    df_OL_agreement[column] = np.where(((df_1OL[column] == df_2OL[column]) & (df_3OL[column] == df_2OL[column]) & (df_3OL[column] == df_1OL[column])), 3, df_OL_agreement[column])
    df_OL_agreement[column] = np.where(((df_1OL[column] != df_2OL[column]) & (df_3OL[column] != df_2OL[column]) & (df_3OL[column] != df_1OL[column])), 0, df_OL_agreement[column])

#df_OL_agreement

# Nerve

In [20]:
columns_drop_nerve = ['General comments', 
                   'Flag this gene as a potential one to use for a figure in the paper?',
                   'Is figure quality too poor?', 
                   'Nascent transcript level', 
                   'Is there obvious RNA/protein discordance (whole image)?', 
                   'Cytosolic RNA level', 
                   'What is the level of protein expression?', 
                   'What is the level of RNA expression?',
                    'Unnamed: 0', 
                    'Is protein distribution cell-type specific?', 
                    'Is RNA distribution cell-type specific?']


df_1nerve, df_2nerve, df_3nerve = threeway_comparer(df_nerve_1, df_nerve_2,  df_nerve_3, 'figure_id', 
                         ['What patterns does the RNA make?',
                          'What patterns does the protein make?'], columns_drop_nerve)

dfanerve = (cohen_kappa_que(df_2nerve, df_3nerve))
dfbnerve = (cohen_kappa_que(df_2nerve, df_1nerve))
dfcnerve = (cohen_kappa_que(df_3nerve, df_1nerve))

dfnerve_mean = (dfanerve + dfbnerve + dfcnerve)/3


df_simple_anerve = simple_agreement(df_3nerve, df_2nerve)
df_simple_bnerve = simple_agreement(df_1nerve, df_2nerve)
df_simple_cnerve = simple_agreement(df_1nerve, df_3nerve)

dfnerve_simple_mean = (df_simple_anerve + df_simple_bnerve + df_simple_cnerve)/3

In [21]:
df_nerve_agreement = df_1nerve.copy()

for column in df_nerve_agreement.columns:

    df_nerve_agreement[column] = np.where(((df_1nerve[column] != df_2nerve[column]) & (df_1nerve[column] == df_3nerve[column])), 2, df_nerve_agreement[column])
    df_nerve_agreement[column] = np.where(((df_1nerve[column] != df_2nerve[column]) & (df_2nerve[column] == df_3nerve[column])), 2, df_nerve_agreement[column])

    df_nerve_agreement[column] = np.where(((df_2nerve[column] != df_3nerve[column]) & (df_2nerve[column] == df_1nerve[column])), 2, df_nerve_agreement[column])
    df_nerve_agreement[column] = np.where(((df_2nerve[column] != df_3nerve[column]) & (df_3nerve[column] == df_1nerve[column])), 2, df_nerve_agreement[column])

    df_nerve_agreement[column] = np.where(((df_1nerve[column] != df_3nerve[column]) & (df_1nerve[column] == df_2nerve[column])), 2, df_nerve_agreement[column])
    df_nerve_agreement[column] = np.where(((df_1nerve[column] != df_3nerve[column]) & (df_3nerve[column] == df_2nerve[column])), 2, df_nerve_agreement[column])

    df_nerve_agreement[column] = np.where(((df_1nerve[column] == df_2nerve[column]) & (df_3nerve[column] == df_2nerve[column]) & (df_3nerve[column] == df_1nerve[column])), 3, df_nerve_agreement[column])
    df_nerve_agreement[column] = np.where(((df_1nerve[column] != df_2nerve[column]) & (df_3nerve[column] != df_2nerve[column]) & (df_3nerve[column] != df_1nerve[column])), 0, df_nerve_agreement[column])

#df_nerve_agreement

In [22]:
df_OL_agreement = df_OL_agreement.add_prefix('Discordant_OL_')
df_NMJ_agreement = df_NMJ_agreement.add_prefix('Discordant_NMJ_')
df_VNC_agreement = df_VNC_agreement.add_prefix('Discordant_VNC_')
df_nerve_agreement = df_nerve_agreement.add_prefix('Discordant_nerve_')
df_MB_agreement = df_MB_agreement.add_prefix('Discordant_MB_')
df_CB_agreement = df_CB_agreement.add_prefix('Discordant_CB_')


In [23]:
dfmrg1 = df_OL_agreement.merge(df, left_index=True, right_on='OL_id')
dfmrg2 = df_NMJ_agreement.merge(dfmrg1, left_index=True, right_on='NMJ_id')
dfmrg3 = df_MB_agreement.merge(dfmrg2, left_index=True, right_on='MB_id')
dfmrg4 = df_CB_agreement.merge(dfmrg3, left_index=True, right_on='CB_id')
dfmrg5 = df_VNC_agreement.merge(dfmrg4, left_index=True, right_on='VNC_id')
dfmrg6 = df_nerve_agreement.merge(dfmrg5, left_index=True, right_on='nerve_glia_id')

In [24]:
dfmrg6.iloc[:, :-7].to_csv('scoring_agreement.csv', index=False)

In [26]:
## Creating cohen Kappa and simple agreement matrix

dfOL_mean = dfOL_mean.T.add_prefix('OL:').T
dfNMJ_mean = dfNMJ_mean.T.add_prefix('NMJ:').T
dfVNC_mean = dfVNC_mean.T.add_prefix('VNC:').T
dfnerve_mean = dfnerve_mean.T.add_prefix('nerve:').T
dfMB_mean = dfMB_mean.T.add_prefix('MB:').T
dfCB_mean = dfCB_mean.T.add_prefix('CB:').T

dfOL_simple_mean = dfOL_simple_mean.T.add_prefix('OL:').T
dfNMJ_simple_mean = dfNMJ_simple_mean.T.add_prefix('NMJ:').T
dfVNC_simple_mean = dfVNC_simple_mean.T.add_prefix('VNC:').T
dfnerve_simple_mean = dfnerve_simple_mean.T.add_prefix('nerve:').T
dfMB_simple_mean = dfMB_simple_mean.T.add_prefix('MB:').T
dfCB_simple_mean = dfCB_simple_mean.T.add_prefix('CB:').T


In [27]:
df_mean = pd.concat([dfOL_mean, dfVNC_mean, dfNMJ_mean, dfMB_mean, dfCB_mean, dfnerve_mean])


In [28]:
df_mean_simple = pd.concat([dfOL_simple_mean, dfVNC_simple_mean, dfNMJ_simple_mean, dfMB_simple_mean, dfCB_simple_mean, dfnerve_simple_mean])
df_mean_simple.reset_index(inplace=True)


In [29]:
df_mean.merge(df_mean_simple, left_index=True, right_on='index').to_csv('metrics_agreement.csv', index=False)